# VISUALIZACIÓN DE DATOS: CLASE Nº 2

# ÍNDICE
- Nº actuals y forecasts.
- Horizonte de previsión.
- Nº Países, productos...
- Histórico de datos de actuals/forecasts.
- Forecasts distintos.

# INTRO
DIVISIÓN DE ACTUALS Y FORECAST

In [38]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose

# Leer el archivo CSV
archivo = "datos_ejercicio_ventas2.csv"
df = pd.read_csv(archivo)

# Filtrar los datos para tener 'actuals' y 'forecasts' separados
df_actuals = df[df['SCENARIO'] == 'actual']
df_forecasts = df[df['SCENARIO'] == 'AI_forecast']



# EJERCICIO 1

In [39]:
# EJERCICIO 1: Distribución de Ventas Realizadas

# 1a. Distribución de ventas por país (un solo color)
fig = px.histogram(df_actuals, x='COUNTRY', y='AMOUNT', title='Distribución de Ventas por País (Actuals)',
                   labels={'COUNTRY': 'País', 'AMOUNT': 'Cantidad de Ventas'}, color_discrete_sequence=['#636EFA'])
fig.update_layout(bargap=0.2, xaxis_title='País', yaxis_title='Cantidad de Ventas')
fig.show()

In [40]:
# 1b. Distribución de ventas por mes y año (un solo color)
df_actuals['YEAR_MONTH'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str))
fig = px.histogram(df_actuals, x='YEAR_MONTH', y='AMOUNT', title='Distribución de Ventas por Mes y Año (Actuals)',
                   labels={'YEAR_MONTH': 'Año-Mes', 'AMOUNT': 'Cantidad de Ventas'}, color_discrete_sequence=['#636EFA'])
fig.update_layout(bargap=0.2, xaxis_title='Año-Mes', yaxis_title='Cantidad de Ventas')
fig.show()

<ipython-input-40-5d891b10070c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
# 1c. Distribución de ventas por submarca (gráfico de pastel)
fig = px.pie(df_actuals, names='SUBBRAND', values='AMOUNT', title='Distribución de Ventas por Submarca (Actuals)')
fig.update_traces(textinfo='percent+label')
fig.show()

# EJERCICIO 2

In [42]:
# EJERCICIO 2: Tendencia y Estacionalidad

# 2a. País con menos ventas
pais_menos_ventas = df_actuals.groupby('COUNTRY')['AMOUNT'].sum().idxmin()
df_pais_menos_ventas = df_actuals[df_actuals['COUNTRY'] == pais_menos_ventas]

# Tendencia de ventas del país con menos ventas
ventas_pais = df_pais_menos_ventas.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
ventas_pais['YEAR_MONTH'] = pd.to_datetime(ventas_pais['YEAR'].astype(str) + '-' + ventas_pais['MONTH'].astype(str))
ventas_pais = ventas_pais.sort_values('YEAR_MONTH')

fig = px.line(ventas_pais, x='YEAR_MONTH', y='AMOUNT', title=f'Tendencia de Ventas en {pais_menos_ventas} (Actuals)',
              labels={'YEAR_MONTH': 'Año-Mes', 'AMOUNT': 'Cantidad de Ventas'})
fig.update_traces(line=dict(color="#EF553B", width=2))
fig.update_layout(xaxis_title='Año-Mes', yaxis_title='Cantidad de Ventas')
fig.show()

In [43]:
# 2b. Submarca con más ventas
marca_mas_ventas = df_actuals.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()
df_marca_mas_ventas = df_actuals[df_actuals['SUBBRAND'] == marca_mas_ventas]

# Tendencia de ventas de la marca con más ventas
ventas_marca = df_marca_mas_ventas.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()
ventas_marca['YEAR_MONTH'] = pd.to_datetime(ventas_marca['YEAR'].astype(str) + '-' + ventas_marca['MONTH'].astype(str))
ventas_marca = ventas_marca.sort_values('YEAR_MONTH')

fig = px.line(ventas_marca, x='YEAR_MONTH', y='AMOUNT', title=f'Tendencia de Ventas para la Marca con Más Ventas: {marca_mas_ventas} (Actuals)',
              labels={'YEAR_MONTH': 'Año-Mes', 'AMOUNT': 'Cantidad de Ventas'})
fig.update_traces(line=dict(color="#00CC96", width=2))
fig.update_layout(xaxis_title='Año-Mes', yaxis_title='Cantidad de Ventas')
fig.show()

# EJERCICIO 3

In [44]:
# EJERCICIO 3: Predicciones hechas en España y análisis de precisión

# Filtrar los datos de España en actuals
df_espana_actuals = df_actuals[df_actuals['COUNTRY'] == 'Spain']

# Crear una columna de tipo fecha combinando 'YEAR' y 'MONTH' para crear un índice de serie temporal
df_espana_actuals['YEAR_MONTH'] = pd.to_datetime(df_espana_actuals['YEAR'].astype(str) + '-' + df_espana_actuals['MONTH'].astype(str))
df_espana_actuals.set_index('YEAR_MONTH', inplace=True)

# Sumar las ventas mensuales para obtener una serie temporal continua
ventas_espana = df_espana_actuals['AMOUNT'].resample('M').sum()

# Descomponer la serie temporal usando seasonal_decompose
descomposicion = seasonal_decompose(ventas_espana, model='additive', period=6)

# Visualización de la descomposición
fig = go.Figure()

# Gráfico de la serie original
fig.add_trace(go.Scatter(x=descomposicion.observed.index, y=descomposicion.observed,
                         mode='lines', name='Original', line=dict(color='blue')))

# Gráfico de la tendencia
fig.add_trace(go.Scatter(x=descomposicion.trend.index, y=descomposicion.trend,
                         mode='lines', name='Tendencia', line=dict(color='orange')))

# Gráfico de la estacionalidad
fig.add_trace(go.Scatter(x=descomposicion.seasonal.index, y=descomposicion.seasonal,
                         mode='lines', name='Estacionalidad', line=dict(color='green')))

# Ajustes de la gráfica
fig.update_layout(title='Descomposición de la Serie Temporal de Ventas en España',
                  xaxis_title='Fecha', yaxis_title='Cantidad de Ventas',
                  template='plotly_white')
fig.show()

# Filtrar los datos de "Pepsi"
df_pepsi_actuals = df_actuals[df_actuals['SUBBRAND'] == 'Pepsi Max (L3)']

# Crear una columna de tipo fecha combinando 'YEAR' y 'MONTH' para crear un índice de serie temporal
df_pepsi_actuals['YEAR_MONTH'] = pd.to_datetime(df_pepsi_actuals['YEAR'].astype(str) + '-' + df_pepsi_actuals['MONTH'].astype(str))
df_pepsi_actuals.set_index('YEAR_MONTH', inplace=True)

# Sumar las ventas mensuales para obtener una serie temporal continua
ventas_pepsi = df_pepsi_actuals['AMOUNT'].resample('M').sum()

# Descomponer la serie temporal usando seasonal_decompose
descomposicion_pepsi = seasonal_decompose(ventas_pepsi, model='additive', period=6)

# Visualización de la descomposición para "Pepsi"
fig = go.Figure()

# Gráfico de la serie original
fig.add_trace(go.Scatter(x=descomposicion_pepsi.observed.index, y=descomposicion_pepsi.observed,
                         mode='lines', name='Original', line=dict(color='blue')))

# Gráfico de la tendencia
fig.add_trace(go.Scatter(x=descomposicion_pepsi.trend.index, y=descomposicion_pepsi.trend,
                         mode='lines', name='Tendencia', line=dict(color='orange')))

# Gráfico de la estacionalidad
fig.add_trace(go.Scatter(x=descomposicion_pepsi.seasonal.index, y=descomposicion_pepsi.seasonal,
                         mode='lines', name='Estacionalidad', line=dict(color='green')))

# Ajustes de la gráfica
fig.update_layout(title='Descomposición de la Serie Temporal de Ventas para la Submarca Pepsi',
                  xaxis_title='Fecha', yaxis_title='Cantidad de Ventas',
                  template='plotly_white')
fig.show()


<ipython-input-44-2f386ef7ee92>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-44-2f386ef7ee92>:11: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



<ipython-input-44-2f386ef7ee92>:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-44-2f386ef7ee92>:45: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [45]:
# Filtrar los datos de España en forecasts (predicciones)
df_espana_forecast = df_forecasts[df_forecasts['COUNTRY'] == 'Spain']

# Crear una columna de tipo fecha combinando 'YEAR' y 'MONTH' para las predicciones
df_espana_forecast['FECHA'] = pd.to_datetime(df_espana_forecast[['YEAR', 'MONTH']].astype(str).agg('-'.join, axis=1), format='%Y-%m')

# Agrupar las predicciones por FECHA y FORECAST, y sumar las ventas
ventas_espana_forecast = df_espana_forecast.groupby(['FECHA', 'FORECAST'])['AMOUNT'].sum().reset_index()

# Crear las series temporales para cada valor único en 'FORECAST'
for forecast_value in ventas_espana_forecast['FORECAST'].unique():
    # Filtrar los datos para el forecast específico
    serie = ventas_espana_forecast[ventas_espana_forecast['FORECAST'] == forecast_value]

    # Crear gráfico de líneas para la serie temporal de ventas mensuales de la predicción
    fig = go.Figure()

    # Línea sólida para las ventas mensuales del forecast específico
    fig.add_trace(go.Scatter(
        x=serie['FECHA'],
        y=serie['AMOUNT'],
        mode='lines+markers',
        name=f'Ventas Mensuales ({forecast_value})',
        line=dict(color='orange', dash='solid')  # Cambié el color de la línea a naranja
    ))

    # Línea punteada para las ventas mensuales generales
    fig.add_trace(go.Scatter(
        x=ventas_pais['YEAR_MONTH'],
        y=ventas_pais['AMOUNT'],
        mode='lines+markers',
        name='Ventas Mensuales Generales',
        line=dict(color='green')  # Cambié el color de la línea a verde
    ))

    # Añadir líneas verticales entre los puntos correspondientes de ambas series
    for fecha in serie['FECHA']:
        # Verificar si la fecha también está presente en la serie de ventas generales
        if fecha in ventas_pais['YEAR_MONTH'].values:
            # Obtener los valores de ventas para ambas series en esa fecha
            valor_forecast = serie.loc[serie['FECHA'] == fecha, 'AMOUNT'].values[0]
            valor_general = ventas_pais.loc[ventas_pais['YEAR_MONTH'] == fecha, 'AMOUNT'].values[0]

            # Añadir línea vertical punteada
            fig.add_trace(go.Scatter(
                x=[fecha, fecha],  # Mantener la misma fecha para línea vertical
                y=[valor_forecast, valor_general],  # Unir el punto de cada serie
                mode='lines',
                line=dict(color='black', dash='dot'),  # Cambié el color de las líneas verticales a negro
                showlegend=False  # Ocultar leyenda para estas líneas
            ))

    # Configurar el título y mostrar el gráfico
    fig.update_layout(
        title=f"Comparación de Predicciones España - {forecast_value}",
        xaxis_title="Fecha",
        yaxis_title="Cantidad Vendida",
    )

    # Mostrar el gráfico
    fig.show()


<ipython-input-45-b650da1bfc1b>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Las previsiones iniciales son bastantes buenas, según nos vamos alejando a predicciones posteriores van siendo bastante peores.